<a href="https://colab.research.google.com/github/Keer-Luo/ISYS5002-Assignment/blob/main/Sandbox_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment Ideas

1. State the problem clearly
2. Describle the input and output information
3. Work the problem
4. Develop an algorithm
5. Test solution with a variety of data

# Inputs

* input a ticker symbol. Not sure what a ticker symbol is?

Note: To prompt the user, I guess I will need to use the input() function

**TODO: Find out what a ticker symbol is** 

# Processing thoughts

* Scrap the Yahoo website
* Extract the 'Key Executives'
* Calculate income tax

Note: In a previous Notebook, we calculated the tax based on ATO table, Might be able to  use that in this assignment

**TODO: Find Yahoo Website**

Here is the webpage: https://finance.yahoo.com/

Note there is a search bar, will use to find a company

**TODO: Find the key Executivies**
# Outputs

* Save the CSV files
* save to SQL database 

**TODO: Work out what I have to save!**


# Lets Explore the Website

1. Search for Coles

Coles ticker is COL.AX

Note the 'AX' means Australian

Just read the assignment specification, it can be any company.

Found the key executivies in the profile page.

Here is the URL for Coles:

https://finance.yahoo.com/quote/COL.AX/profile?p=COL.AX

2. Lets search for National Australian Back

The ticker is NAB.AX, Here is the URL to the proflie page:

https://finance.yahoo.com/quote/NAB.AX/profile?p=NAB.AX

There appears to be a pattern with the URL. I can see that the ticker twice in the URL

Manual process is
1. Seaech for Ticker
2. Click on proflie

**TODO: Can I shortcut the process and just use the URL**

Looks like I can edit the URL and replace the ticker. Seems that only the first ticker symbol in the URL is important, but to be safe I will replace both.




# Initial Algorithm

1. User input ticker symbol
2. Use ticker to get profile page
3. Copied the table
4. For each row of the table
 * Process a row to extract pay
 * Find the last character to determin multiplier
 * Calculate salary, pay * multiplier
 * Calculate the income tax


Note: 
1. Need to validate the ticker, but for now I will assume the user type in a correct ticker
2. Still need to work out where I save the income tax to


In [1]:

#install requirements
!pip install kora -q

#load package page
from bs4 import BeautifulSoup
from kora.selenium import wd
import pandas as pd

     |████████████████████████████████| 57 kB 3.5 MB/s 
     |████████████████████████████████| 56 kB 4.0 MB/s 


In [7]:
def build_url():
 # User input ticker symbol
 ticker_symbol = input("Please input ticker symbol ")
 print(ticker_symbol)
 # Use ticker to get profile page
 example_url = 'https://au.finance.yahoo.com/quote/{}/profile?p={}'
 result = example_url.format(ticker_symbol, ticker_symbol)
 return result

In [17]:
def get_first_3_cells(row):
  cells = row.find_all('td')
  name = cells[0].text
  title = cells[1].text
  pay = cells[2].text
  return {'name': name, 'title':title, 'pay': pay}

In [25]:
def get_exec_records(soup):
  exec_records =[]
  for row in rows:
   result = get_first_3_cells(row)
   exec_records.append(result) 
  return exec_records

In [28]:
# Get the URL
url = build_url()
wd.get(url)
soup = BeautifulSoup(wd.page_source, "html.parser")
get_exec_records()

Please input ticker symbol nab.ax
nab.ax


[{'name': 'Mr. Ross Maxwell McEwan CBE, B.B.S.',
  'pay': '2.54M',
  'title': 'Group CEO, MD & Director'},
 {'name': 'Mr. Gary Andrew Lennon BEC, F.C.A., BEc (Hons), FCA',
  'pay': '1.13M',
  'title': 'Group Chief Financial Officer'},
 {'name': 'Mr. Patrick F. Wright',
  'pay': '1.72M',
  'title': 'Group Exec. of Technology & Enterprise Operations'},
 {'name': 'Ms. Sharon J. Cook',
  'pay': '912.98k',
  'title': 'Group Exec. of Legal & Commercial Services'},
 {'name': 'Ms. Susan  Ferrier',
  'pay': '929.27k',
  'title': 'Group Exec. of People & Culture'},
 {'name': 'Ms. Angela  Mentis B.Bus',
  'pay': '1.62M',
  'title': 'Group Chief Digital, Data & Analytics Officer'},
 {'name': 'Mr. Shaun  Dooley', 'pay': '1.05M', 'title': 'Chief Risk Officer'},
 {'name': 'Mr. Andrew  Irvine',
  'pay': '333.27k',
  'title': 'Chief Exec. of Bus. Private Banking Division'},
 {'name': 'Mr. Lesile D. Matheson',
  'pay': 'N/A',
  'title': 'Group Chief Operating Officer'},
 {'name': 'Ms. Sally  Mihell',
  